## Setup 1

In [2]:
!pip install --upgrade --quiet openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 8.1 MB/s eta 0:00:00


In [4]:
import os
from openai import OpenAI
from google.colab import userdata
key = userdata.get('OPENAI_API_KEY')

In [5]:
client = OpenAI(api_key=key)

# Exercise 1: Sentiment Analysis and Moderation

## Q1: Sentiment Analysis


In [8]:
sentiment_data = [
    "I had an amazing experience at the new restaurant downtown!",
    "The service was terrible, and I will never go back.",
    "It was an okay movie, nothing special.",
    "The product quality is outstanding, I highly recommend it.",
    "I'm extremely disappointed with my purchase.",
    "The weather is nice today."
]

In [16]:
def analyze_sentiment_openai(text):
    prompt = f"Please analyze the sentiment of this text: '{text}'. Is it positive, negative, or neutral? And give me the reasoning."
    messages = [
        {"role": "user", "content": prompt},
        {"role": "system", "content": "Give me the response in the JSON format which looks like this: {Text: \"\", Sentiment: \"\", Reasoning: \"\"}"}
    ]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=1,
    )

    # Attempt to parse the response as JSON
    try:
        sentiment_result = json.loads(response.choices[0].message.content)
        # Add the original text to the result
        sentiment_result["Text"] = text
        return sentiment_result
    except json.JSONDecodeError:
        # Handle cases where the response is not valid JSON
        return {
            "Text": text,
            "Sentiment": "Error",
            "Reasoning": "Could not parse response as JSON"
        }
    return sentiment

In [18]:
results = []

for text in sentiment_data:
    sentiment_result = analyze_sentiment_openai(text)  # Get sentiment with reasoning

    results.append(sentiment_result)

# Display the results
import json
print(json.dumps(results, indent=2))

[
  {
    "Text": "I had an amazing experience at the new restaurant downtown!",
    "Sentiment": "Positive",
    "Reasoning": "The use of the word 'amazing' indicates a strong positive feeling towards the experience at the restaurant. Additionally, the phrase 'new restaurant downtown' suggests excitement and novelty, further contributing to the overall positive sentiment."
  },
  {
    "Text": "The service was terrible, and I will never go back.",
    "Sentiment": "negative",
    "Reasoning": "The text expresses dissatisfaction with the service by using the word 'terrible,' which conveys strong discontent. Additionally, the phrase 'I will never go back' indicates a definitive decision to avoid returning, reinforcing the negative sentiment."
  },
  {
    "Text": "It was an okay movie, nothing special.",
    "Sentiment": "Error",
    "Reasoning": "Could not parse response as JSON"
  },
  {
    "Text": "The product quality is outstanding, I highly recommend it.",
    "Sentiment": "Error"

## Q2: Content Moderation


In [19]:
content_moderation_data = [
    "I can't believe they hired such an incompetent person!",
    "This is a wonderful community, and I love being part of it.",
    "The event was a disaster; the organizers did a horrible job.",
    "You are such an idiot for thinking that!",
    "I support everyone who works hard to achieve their dreams.",
    "Get lost, no one wants you here!"
]

In [42]:
import json

def moderate_content_openai(text):
    prompt = f"""Please check the following text for harmful, offensive, or inappropriate content: '{text}'.

    If there is inappropriate content, suggest a moderation action.

    Provide your response in the following JSON format:

    {{
      "Text": "original_text_here",
      "Identified Issue": "issue_description_here",
      "Moderation Action": "action_to_take_here",
      "Reasoning": "reasoning_behind_the_action_here"
    }}
    """
    messages = [{"role": "user", "content": prompt}]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=1,
    )

    try:
        moderate_content = json.loads(response.choices[0].message.content)
        return moderate_content
    except json.JSONDecodeError:
        # Handle cases where the response is not valid JSON
        return {
            "Text": text,
            "Identified Issue": "Error",
            "Moderation Action": "Error",
            "Reasoning": "Could not parse response as JSON"
        }

In [44]:
results = []

# Loop through each text for content moderation
for text in content_moderation_data:
    moderation_result = moderate_content_openai(text)  # Get moderation result

    results.append(moderation_result)

# Display the results in the required JSON format
print(json.dumps(results, indent=2))


[
  {
    "Text": "I can't believe they hired such an incompetent person!",
    "Identified Issue": "Offensive language towards an individual or group",
    "Moderation Action": "Issue a warning or request for rephrasing",
    "Reasoning": "The statement uses derogatory language that could be seen as disrespectful or harmful, suggesting a judgment on someone's competence in a negative way. Encouraging more respectful language can foster a better dialogue."
  },
  {
    "Text": "This is a wonderful community, and I love being part of it.",
    "Identified Issue": "None",
    "Moderation Action": "No action needed",
    "Reasoning": "The text expresses a positive sentiment about a community and contains no harmful, offensive, or inappropriate content."
  },
  {
    "Text": "The event was a disaster; the organizers did a horrible job.",
    "Identified Issue": "Negative criticism of organizers",
    "Moderation Action": "Flag for review",
    "Reasoning": "The language used is highly crit

# Exercise 2: Few-shot Prompting

## Q1: Text Classification


In [54]:
# Few-shot prompt to classify text into Factual, Opinion, or Ambiguous
def classify_text(text):
    prompt = """
    Classify the following statements into one of the following categories: Factual, Opinion, or Ambiguous.

    Examples:
    "The Earth is the third planet from the Sun." -> Factual
    "I believe chocolate is the best dessert." -> Opinion
    "It might rain tomorrow." -> Ambiguous
    "Mount Everest is the highest mountain in the world." -> Factual
    "In my opinion, summer is the best season." -> Opinion
    "She may arrive at the party tonight." -> Ambiguous
    "Pizza is delicious." -> Opinion
    "The Earth orbits around the Sun." -> Factual
    "Sunsets are beautiful." -> Opinion
    "Water boils at 100 degrees Celsius." -> Factual
    "Classical music is soothing." -> Opinion
    "The meeting could be postponed." -> Ambiguous
    "Football is the most popular sport globally." -> Opinion
    "She could decide to travel abroad." -> Ambiguous
    "Ice cream is better than cake." -> Opinion

    Now, classify the following text:
    """ + text

    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=1,
    )
    classification = response.choices[0].message.content.strip()
    return classification

In [55]:
# List of texts to classify
texts_to_classify = [
    "The capital of France is Paris.",
    "I believe chocolate is the best dessert.",
    "It might rain tomorrow.",
    "Mount Everest is the highest mountain in the world.",
    "In my opinion, summer is the best season.",
    "She may arrive at the party tonight.",
    "Pizza is delicious.",
    "The Earth orbits around the Sun.",
    "Sunsets are beautiful.",
    "Water boils at 100 degrees Celsius.",
    "Classical music is soothing.",
    "The meeting could be postponed.",
    "Football is the most popular sport globally.",
    "She could decide to travel abroad.",
    "Ice cream is better than cake."
]

In [56]:
# Initialize the result list
results = []

# Loop through each text and classify
for text in texts_to_classify:
    label = classify_text(text)  # Get classification label
    result = {
        "text": text,
        "label": label
    }
    results.append(result)

# Display the results in JSON format
import json
print(json.dumps(results, indent=2))

[
  {
    "text": "The capital of France is Paris.",
    "label": "Factual"
  },
  {
    "text": "I believe chocolate is the best dessert.",
    "label": "Opinion"
  },
  {
    "text": "It might rain tomorrow.",
    "label": "Ambiguous"
  },
  {
    "text": "Mount Everest is the highest mountain in the world.",
    "label": "Factual"
  },
  {
    "text": "In my opinion, summer is the best season.",
    "label": "Opinion"
  },
  {
    "text": "She may arrive at the party tonight.",
    "label": "Ambiguous"
  },
  {
    "text": "Pizza is delicious.",
    "label": "Opinion"
  },
  {
    "text": "The Earth orbits around the Sun.",
    "label": "Factual"
  },
  {
    "text": "Sunsets are beautiful.",
    "label": "Opinion"
  },
  {
    "text": "Water boils at 100 degrees Celsius.",
    "label": "Factual"
  },
  {
    "text": "Classical music is soothing.",
    "label": "Opinion"
  },
  {
    "text": "The meeting could be postponed.",
    "label": "Ambiguous"
  },
  {
    "text": "Football i

## Q2: Logical Reasoning


In [51]:
# Few-shot prompt to identify the conclusion based on the premises
def identify_conclusion(premises):
    prompt = """
    Identify the conclusion from the following premises:

    Example 1:
    Premises: "All birds have wings. A sparrow is a bird."
    Conclusion: "A sparrow has wings"

    Example 2:
    Premises: "If it snows, the ground will be covered in snow. It is snowing."
    Conclusion: "The ground is covered in snow"

    Example 3:
    Premises: "All dogs are mammals. Max is a dog."
    Conclusion: "Max is a mammal"

    Now, based on the following premises, identify the conclusion,
    note: conclusion you return should be a pure text actual conclusion nothing else:
    """ + premises

    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=1,
    )
    conclusion = response.choices[0].message.content.strip()
    return conclusion

In [52]:
# List of test cases with premises
premises_data = [
    "If it rains, the streets will be wet. It is raining.",
    "All squares are rectangles. This shape is a square.",
    "If it is sunny, she will go for a walk. It is sunny.",
    "All students must pass the final exam. John is a student.",
    "If the oven is turned on, the food will cook. The oven is turned on.",
    "All prime numbers are odd. Seven is a prime number.",
    "If he studies hard, he will pass the test. He studied hard.",
    "All mammals have hair. Whales are mammals.",
    "If it is cold outside, she wears a jacket. It is cold outside.",
    "All metals conduct electricity. Copper is a metal."
]


In [53]:
# Initialize the result list
results = []

# Loop through each test case and identify the conclusion
for premises in premises_data:
    conclusion = identify_conclusion(premises)  # Get conclusion based on premises
    result = {
        "premises": premises,
        "conclusion": conclusion
    }
    results.append(result)

# Display the results in JSON format
import json
print(json.dumps(results, indent=2))

[
  {
    "premises": "If it rains, the streets will be wet. It is raining.",
    "conclusion": "The streets will be wet."
  },
  {
    "premises": "All squares are rectangles. This shape is a square.",
    "conclusion": "This shape is a rectangle."
  },
  {
    "premises": "If it is sunny, she will go for a walk. It is sunny.",
    "conclusion": "She will go for a walk."
  },
  {
    "premises": "All students must pass the final exam. John is a student.",
    "conclusion": "John must pass the final exam."
  },
  {
    "premises": "If the oven is turned on, the food will cook. The oven is turned on.",
    "conclusion": "The food will cook."
  },
  {
    "premises": "All prime numbers are odd. Seven is a prime number.",
    "conclusion": "Seven is odd."
  },
  {
    "premises": "If he studies hard, he will pass the test. He studied hard.",
    "conclusion": "He will pass the test."
  },
  {
    "premises": "All mammals have hair. Whales are mammals.",
    "conclusion": "Whales have hair

## Q3: Evaluating your results


In [59]:
classification_output = [
  {
    "text": "The capital of France is Paris.",
    "label": "Factual"
  },
  {
    "text": "I believe chocolate is the best dessert.",
    "label": "Opinion"
  },
  {
    "text": "It might rain tomorrow.",
    "label": "Ambiguous"
  },
  {
    "text": "Mount Everest is the highest mountain in the world.",
    "label": "Factual"
  },
  {
    "text": "In my opinion, summer is the best season.",
    "label": "Opinion"
  },
  {
    "text": "She may arrive at the party tonight.",
    "label": "Ambiguous"
  },
  {
    "text": "Pizza is delicious.",
    "label": "Opinion"
  },
  {
    "text": "The Earth orbits around the Sun.",
    "label": "Factual"
  },
  {
    "text": "Sunsets are beautiful.",
    "label": "Opinion"
  },
  {
    "text": "Water boils at 100 degrees Celsius.",
    "label": "Factual"
  },
  {
    "text": "Classical music is soothing.",
    "label": "Opinion"
  },
  {
    "text": "The meeting could be postponed.",
    "label": "Ambiguous"
  },
  {
    "text": "Football is the most popular sport globally.",
    "label": "Opinion"
  },
  {
    "text": "She could decide to travel abroad.",
    "label": "Ambiguous"
  },
  {
    "text": "Ice cream is better than cake.",
    "label": "Opinion"
  }
]

reasoning_output = [
  {
    "premises": "If it rains, the streets will be wet. It is raining.",
    "conclusion": "The streets will be wet."
  },
  {
    "premises": "All squares are rectangles. This shape is a square.",
    "conclusion": "This shape is a rectangle."
  },
  {
    "premises": "If it is sunny, she will go for a walk. It is sunny.",
    "conclusion": "She will go for a walk."
  },
  {
    "premises": "All students must pass the final exam. John is a student.",
    "conclusion": "John must pass the final exam."
  },
  {
    "premises": "If the oven is turned on, the food will cook. The oven is turned on.",
    "conclusion": "The food will cook."
  },
  {
    "premises": "All prime numbers are odd. Seven is a prime number.",
    "conclusion": "Seven is odd."
  },
  {
    "premises": "If he studies hard, he will pass the test. He studied hard.",
    "conclusion": "He will pass the test."
  },
  {
    "premises": "All mammals have hair. Whales are mammals.",
    "conclusion": "Whales have hair."
  },
  {
    "premises": "If it is cold outside, she wears a jacket. It is cold outside.",
    "conclusion": "She wears a jacket."
  },
  {
    "premises": "All metals conduct electricity. Copper is a metal.",
    "conclusion": "Copper conducts electricity."
  }
]


In [63]:
from nltk.translate.bleu_score import sentence_bleu

def evaluate_bleu_score(predictions, ground_truths):
    correct = 0
    total = len(predictions)
    bleu_scores = []

    for pred, gt_text in zip(predictions, ground_truths):
        pred_text = pred["label"].split()  # Access "label" and split
        gt_text = gt_text.split()       # Split ground truth string
        bleu_scores.append(sentence_bleu([gt_text], pred_text)) # Provide a list of reference sentences
        if pred["label"] == gt_text[0]:  # Compare with the first word of ground truth (adjust if needed)
            correct += 1

    avg_bleu_score = sum(bleu_scores) / len(bleu_scores) if len(bleu_scores) > 0 else 0

    return correct, total, avg_bleu_score

def evaluate_reasoning(predictions, ground_truths):
    correct = 0
    total = len(predictions)
    bleu_scores = []

    for pred, premises in zip(predictions, ground_truths):
        pred_text = pred["conclusion"].split()  # Split the prediction into words
        # Extract the expected conclusion and split into words
        expected_conclusion = premises.split('.')[-1].strip()
        gt_text = expected_conclusion.split()
        bleu_scores.append(sentence_bleu([gt_text], pred_text)) # Provide list of reference sentences (one in this case)

        if pred["conclusion"] == expected_conclusion:
            correct += 1

    avg_bleu_score = sum(bleu_scores) / len(bleu_scores) if len(bleu_scores) > 0 else 0

    return correct, total, avg_bleu_score

# Assuming you have classification_output, texts_to_classify, reasoning_output, and premises_data

# Modify texts_to_classify to include only ground truth labels
ground_truth_labels = [{"label": text.split("->")[-1].strip()} for text in texts_to_classify]

# Evaluate sentiment analysis
sentiment_correct, sentiment_total, sentiment_bleu = evaluate_bleu_score(classification_output, [t["label"] for t in ground_truth_labels])
# Evaluate reasoning
reasoning_correct, reasoning_total, reasoning_bleu = evaluate_reasoning(reasoning_output, premises_data)
# Calculate overall evaluation results
total_correct = sentiment_correct + reasoning_correct
total_predictions = sentiment_total + reasoning_total
overall_accuracy = total_correct / total_predictions

# Output the results
result = {
    "Total_num_text": total_predictions,
    "Total_correct_predictions": total_correct,
    "Total_incorrect_predictions": total_predictions - total_correct,
    "overall_accuracy": overall_accuracy
}

# Display the results
import json
print(json.dumps(result, indent=2))

{
  "Total_num_text": 25,
  "Total_correct_predictions": 0,
  "Total_incorrect_predictions": 25,
  "overall_accuracy": 0.0
}


# Exercise 3: Chain-of-Thoughts(CoT) Prompting


## Q1: Chain-of-Thoughts(CoT) Prompting

In [ ]:
import openai

# Replace with your OpenAI API key
openai.api_key = 'YOUR_API_KEY'

def generate_trip_plan():
    # Define the Chain-of-Thoughts (CoT) prompt
    prompt = """
    You are tasked with planning a 7-day cultural trip to India, focusing on cultural sites. Break down the task into manageable steps:
    1. Select cities in India that are rich in cultural heritage.
    2. Identify key cultural sites in those cities.
    3. Plan a 7-day itinerary based on the cultural significance of each site.

    Think step by step and explain the reasoning behind each decision. You will plan a full itinerary with the following details:
    - Choose the cities to visit.
    - Identify the major cultural sites in those cities.
    - Plan the daily activities and suggest sites to visit each day.

    At the end, provide the final 7-day itinerary.
    """

    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=1,
    )
    trip_plan = response.choices[0].message.content.strip()

    # Extract and return the trip plan from the model response
    return trip_plan

# Generate the trip plan
trip_plan = generate_trip_plan()

# Print the output (the 7-day itinerary)
print(trip_plan)


Planning a 7-day cultural trip to India involves selecting cities renowned for their rich cultural heritage, identifying key cultural sites, and creating a daily itinerary that maximizes engagement with these cultural landmarks. India’s diverse history, art, and traditions provide countless options, but for a well-rounded experience, we will focus on three cities: Delhi, Agra, and Jaipur, collectively known as the Golden Triangle.

### Step 1: Select Cities

**Rationale**: The Golden Triangle is a popular route for cultural tourism in India. It connects Delhi (the capital city), Agra (home to the Taj Mahal), and Jaipur (the Pink City), showcasing a rich tapestry of historical and cultural attractions.

### Selected Cities:
1. **Delhi**
2. **Agra**
3. **Jaipur**

### Step 2: Identify Key Cultural Sites

**Delhi**:
- Red Fort: A UNESCO World Heritage Site symbolizing Mughal architecture.
- Qutub Minar: Another UNESCO site, this minaret is one of the tallest in India.
- Humayun's Tomb: Th